# Lesson 3: Chains in LangChain

## What is a chain in LangChain ?
A chain is essentially a sequence of operations that takes input, processes it through various components, and generates an output. Chains are the building blocks for creating workflows that integrate language models, tools, and logic to perform tasks.

### Key Features of Chains: 
1. **Modularity**: Chains consist of discrete components, like prompts, memory, tools, and other chains, that can be connected together. 
2. **Input/Output Transformation**: Chains take structured inputs (e.g., dictionaries) and return structured outputs. They abstract away the complexity of managing intermediate states.
3. **Composability**: Chains can be nested to build more complex workflows, where the output of one chain becomes the input to another.
4. **Integration**: Chains can integrate with external APIs, databases, or other computational tools to fetch, process, or store data.

### When to Use Chains:
* Automating workflows: For repetitive tasks, like text summarization or question answering.
* Managing complexity: When building multi-step pipelines or applications.
* Interactive systems: For chatbots, virtual assistants, or systems that need memory/context.


### Outline of lesson
* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

## LLMChain

The class `LLMChain` is deprecated in the Langchain, so we have to use the pipe command: `|` to chain the prompt and the LLM model 

In [ ]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

llm = ChatOllama(model="llama3.2", temperature=0)

prompt = ChatPromptTemplate.from_template( "What is the best name to describe \
    a company that makes {product}?")

chain = prompt | llm #This is of type RunnableSequence, same as chain = RunnableSequence(first=runnable_1, last=runnable_2)
chain.invoke("Lazer Gun")


AIMessage(content='Here are some suggestions for a company name that makes laser guns:\n\n1. **LaserStrike Inc.**: This name conveys a sense of power and precision, suggesting a company that produces high-quality laser technology.\n2. **Apex Laser Systems**: "Apex" implies a pinnacle of achievement, which could be fitting for a company that creates cutting-edge laser guns.\n3. **Lumina Laser Technologies**: "Lumina" means light in Latin, which is fitting for a laser-based product. This name also has a futuristic and innovative feel to it.\n4. **Pulse Laser Solutions**: "Pulse" suggests energy and movement, which could be appealing for a company that produces dynamic laser technology.\n5. **Spectra Laser Systems**: "Spectra" refers to the range of colors or frequencies in light, which is fitting for a laser-based product. This name also has a scientific and technical feel to it.\n6. **NovaTech Laser**: "Nova" means new in Latin, which could suggest innovation and progress. This name als

In [2]:
print(chain)
print(type(chain))  

first=ChatPromptTemplate(input_variables=['product'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product'], input_types={}, partial_variables={}, template='What is the best name to describe     a company that makes {product}?'), additional_kwargs={})]) middle=[] last=ChatOllama(model='llama3.2', temperature=0.0)
<class 'langchain_core.runnables.base.RunnableSequence'>


## SimpleSequentialChain/RunnableSequence
These are linear workflows that process input through one or more steps sequentially.\
Note that when a task is simple, and stateless,i.e, the intermediate results are of no use, we can use a single prompt, rather than chaining it.

Sequential chains are more helpful for **dynamic inputs**: If the output of one step needs to be reused, modified, or combined with other data before being passed to the next step, splitting into chains is more manageable

In [7]:
from langchain.chains.sequential import SimpleSequentialChain
from langchain_core.runnables import RunnableSequence

first_prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")
chain_one = first_prompt | llm

second_prompt = ChatPromptTemplate.from_template("Write a 20 words description for the following company:{company_name}")
chain_two = second_prompt | llm

simple_chain = RunnableSequence(first=chain_one, last=chain_two)
# simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

result = simple_chain.invoke("Lazer Gun")
print(result.content)

Here is a 20-word description for the company: "LaserTech Industries - Developing cutting-edge laser guns with precision and innovation for various applications."


## SequentialChain

### Key Differences

| Feature                   | `SequentialChain`                       | `SimpleSequentialChain`              |
|---------------------------|------------------------------------------|---------------------------------------|
| **Flexibility**           | High: Supports input/output mapping.    | Low: Outputs flow linearly.          |
| **Intermediate Outputs**  | Yes: Tracks intermediate results.       | No: Only the final result is stored. |
| **Input/Output Control**  | Customizable per chain step.            | Automatic linear flow.               |
| **Use Case Complexity**   | For complex, multi-step workflows.      | For simple, linear workflows.        |
| **Ease of Use**           | Requires more setup and configuration.  | Easier to implement.                 |

With SequentialChain, at each intermediate step, we can specify where this step can get its input from, doesn't have to be the immediate previous step

Now, in bellow implementation, I have ommittted the use of deprecated `LLMChain`, and used `RunnableSequence`. \
\
`RunnableLambda` is essential here, since after each step in the chain, the output from the LLM is of type `AIMessage`, or `str` in my case due `StrOutputParser`. This doesn't work,
since subsequent prompt template expect a mapped input, i.e key-value pair of input variable and its corresponding value. \
\
I've acheived that here, using `ChatPromptTemplate` object's `format_messages()`method, which I can call using a lambda function inside the LLM chain, as its wrapped with `RunnableLambda`.\
Then, in the middle section of `sequential_chain`, `RunnableMap` allows to explicitly map the outputs from the specified chains to a desired key, so that these outputs can be passed to the last section, and \
here, the "summary" and "language" outputs, are both fed in the last step. If this wasn't used, the output from middle would only contain `qsn_prompt`'s  output, due to the sequential nature.

In [ ]:
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableMap
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

# Define the language model
llm = ChatOllama(model="llama3.2", temperature=0)

# Define prompts
translate_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English:\n\n{Review}"
)
summarize_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:\n\n{English_Review}"
)
qsn_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
followup_prompt = ChatPromptTemplate.from_template(
    "Write a follow-up response to the following summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# Construct the chain
sequential_chain = RunnableSequence(
    first=RunnableLambda( 
            lambda input_prompt: translate_prompt.format_messages(Review=input_prompt)
        ) | llm | StrOutputParser()
        | RunnableLambda(lambda output: print(f"Translation Step Output: {output}") or output),

    middle=[
        RunnableMap({
            "summary": RunnableLambda( lambda output: summarize_prompt.format_messages(English_Review=output) )
            | llm | StrOutputParser() 
            | RunnableLambda(lambda output: print(f"Summarization Step Output: {output}") or output),

            "language": RunnableLambda( lambda output: qsn_prompt.format_messages(Review=output) )
            | llm | StrOutputParser() 
            | RunnableLambda(lambda output: print(f"Language Detection Step Output: {output}") or output)
        })
    ],

    last=RunnableLambda(lambda inputs: print(f"Input to follow_up: {inputs} {type(inputs)}") or inputs) | RunnableLambda(
        lambda outputs: followup_prompt.format_messages(
            summary=outputs["summary"], language=outputs["language"]
        )
    )| llm | StrOutputParser()

)

# Input data
input_data = {
    "Review": "Cette course d'Andrew Ng est une désastre. Je ne recommande pas. Ça détruit mes attentes, car dans chaque leçon, rien n'est expliqué clairement, en plus, "
              "tous les commandes sont obsolètes. Donc, il me faut que je dois chercher sur les docs de Langchain la propre méthode, et pour chaque leçon, ça perd du grand temps. Je suis très déçu."
}

# Invoke the chain
result = sequential_chain.invoke(input_data["Review"])
print(result)


Translation Step Output: Here's the translation of the review to English:

"This Andrew Ng course is a disaster. I do not recommend it. It shattered my expectations because in every lesson, nothing is explained clearly, and on top of that, all the commands are outdated. So, I have to spend time looking up the methods from Langchain documentation for each lesson, which wastes so much time. I'm very disappointed."

Note: The original text uses some informal language and colloquial expressions (e.g., "Ça détruit mes attentes", "Je suis très déçu") that are not typically used in formal reviews or written feedback.
Summarization Step Output: The reviewer expresses strong disappointment with Andrew Ng's course, citing unclear explanations, outdated commands, and wasted time researching alternative methods to keep up with the material.
Language Detection Step Output: The review appears to be written in French, as indicated by the use of French phrases such as "Ça détruit mes attentes" (which 

## RouterChain

In [24]:
from langchain_ollama import ChatOllama
from  langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

llm = ChatOllama(model="llama3.2", temperature=0.0)

Configuring the router:

In [28]:
router_template_str = """Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a 
description of what the prompt is best suited for. You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```
REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.
REMEMBER: Only give output that you are being asked for. Don't include any additional information or context or suggestions, just the output.
<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{input}

<< OUTPUT (remember to include the ```json)>>"""

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
C:\Users\milap\AppData\Local\Temp\ipykernel_15528\2724433427.py:1: SyntaxWarning: invalid escape sequence '\ '
  router_template_str = """Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a


And now the destination chains, chains where the `router_chain` will route to: 

In [7]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics in a concise and easy to understand manner.",
        "template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions.",
        "template": math_template,
    },
    {
        "name": "history",
        "description": "Good for answering questions about history.",
        "template": history_template
    },
    {
        "name": "computerscience",
        "description": "Good for answering coding or computer science related questions.",
        "template": computerscience_template
    },
]

In [19]:
destination_prompt_templates = {}
destinations = ""

for prompt_info in prompt_infos:
    name = prompt_info["name"]
    chain = ChatPromptTemplate.from_template(prompt_info["template"])
    destination_prompt_templates[name] = chain

destinations = "\n".join([f"{prompt_info['name']}"for prompt_info in prompt_infos])
print(destinations)

default_prompt_template = ChatPromptTemplate.from_template("You are a very smart person. You are great at answering questions in a concise and easy to understand manner. Here is a question: {input}")
destination_prompt_templates["DEFAULT"] =  default_prompt_template

router_prompt = PromptTemplate(template=router_template_str, input_variables=["input","destinations"])

print(destination_prompt_templates["physics"])
print(router_prompt)

physics
math
history
computerscience
input_variables=['input'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}"), additional_kwargs={})]
input_variables=['destinations', 'input'] input_types={} partial_variables={} template='Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a \ndescription of what the prompt is best suited for. You may also revise the original input if you think that revising\nit will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON objec

In [31]:
from langchain_core.runnables import RunnableSequence
main_chain = RunnableSequence(

    first=RunnableLambda(
        lambda input_prompt: router_prompt.format(
            input=input_prompt, destinations=destinations)
    ) | RunnableLambda(lambda router_stuff: print(f"Input to router chain: \n{router_stuff}\n") or router_stuff)
    | llm | JsonOutputParser(),

    last=RunnableLambda(lambda router_stuff: print(f"Input to destination chain: \n{router_stuff} \n{type(router_stuff)}\n") or router_stuff) |
    RunnableLambda(  
        # Use the destination chain if it exists, otherwise use the default chain
        # router_outputs is a JSON object with keys "destination" and "next_inputs"
        lambda router_outputs:
        destination_prompt_templates[router_outputs["destination"]].format_messages(
            input=router_outputs["next_inputs"]
        )
        if router_outputs["destination"] in destination_prompt_templates else destination_prompt_templates["DEFAULT"]
    )
    | llm | StrOutputParser()
)

In [33]:
answer = main_chain.invoke("What is quantum entaglement?")
print(answer)

Input to router chain: 
Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a 
description of what the prompt is best suited for. You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics
math
history
computerscience

<< INPUT >>
What is quantum entaglement

In [34]:
answer = main_chain.invoke("What are partial differntial equations?")
print(answer)

Input to router chain: 
Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a 
description of what the prompt is best suited for. You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics
math
history
computerscience

<< INPUT >>
What are partial differntia

In [35]:
answer = main_chain.invoke("What is the capital of France?")
print(answer)

Input to router chain: 
Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a 
description of what the prompt is best suited for. You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics
math
history
computerscience

<< INPUT >>
What is the capital of Fran